In [8]:
import ROOT
import numpy as np

In [9]:
# Make a ROOT file with a TTree
f = ROOT.TFile('test.root', 'recreate' )
t = ROOT.TTree('t1', 'tree' )

n = np.zeros(1, dtype='float')  # numpy float is C++ double
m = np.zeros(1, dtype='float')
t.Branch('val1', n, 'val1/D')
t.Branch('val2', m, 'val2/D')

for ii in range(3):
    n[0] = (ii+1)*2
    m[0] = (ii+1)*3
    t.Fill()

f.Write()
f.Close()

In [10]:
# Read in the ROOT file, compute a quantity and write it to a "Friend" tree
f1 = ROOT.TFile('test.root')
t1 = f1.Get('t1')
t1.Scan()

f2 = ROOT.TFile('testfriend.root', 'RECREATE')
t2 = ROOT.TTree('t1f', 'a friend tree')
w = np.zeros(1, dtype='float')
t2.Branch('val3', w, 'val3/D')

for event in t1:
    # Now you have acess to the leaves/branches of each entry in the tree, e.g.
    w[0] = event.val1 + event.val2
    print "%d, %d, %d" % (event.val1, event.val2, w)
    t2.Fill()

t2.Scan()
f2.cd()
t2.Write()
f1.Close()
f2.Close()

************************************
*    Row   *      val1 *      val2 *
************************************
*        0 *         2 *         3 *
*        1 *         4 *         6 *
*        2 *         6 *         9 *
************************************
2, 3, 5
4, 6, 10
6, 9, 15
************************
*    Row   *      val3 *
************************
*        0 *         5 *
*        1 *        10 *
*        2 *        15 *
************************


In [22]:
# Read in the original TTree
fA = ROOT.TFile('test.root')
tA = fA.Get('t1')
tA.Scan()

# And associate the Friend TTree
tA.AddFriend("t1f", 'testfriend.root')

# Then the original and friend are tied together
tA.Scan('val1:val2:val3')

3L

************************************
*    Row   *      val1 *      val2 *
************************************
*        0 *         2 *         3 *
*        1 *         4 *         6 *
*        2 *         6 *         9 *
************************************
************************************************
*    Row   *      val1 *      val2 *      val3 *
************************************************
*        0 *         2 *         3 *         5 *
*        1 *         4 *         6 *        10 *
*        2 *         6 *         9 *        15 *
************************************************
